# MLflow: download, register, and load `.skops` model
This notebook shows how to fetch the best run artifact, re-log/register it as an MLflow model, and load it back.

In [ ]:
import mlflow
import pandas as pd
import skops.io as sio
from pathlib import Path

# Set tracking URI to the same store used during training
tracking_uri = "file:./mlruns"  # change if you used a remote server
mlflow.set_tracking_uri(tracking_uri)
experiment = "money-scam-mm"
mlflow.set_experiment(experiment)

## Find the best run (tagged `best_run=true`)
Update filters if you tag differently.

In [ ]:
runs = mlflow.search_runs(
    experiment_names=[experiment],
    filter_string="tags.best_run = 'true'",
    order_by=["attributes.start_time DESC"],
)
runs[["run_id","run_name","metrics.weighted_f1_test","metrics.weighted_f1_cv_mean"]].head()

## Download and load the `.skops` artifact from a run
Set `run_id` and `artifact_path` if your file is named differently.

In [ ]:
# Choose the run to use (e.g., first best_run)
run_id = runs.iloc[0].run_id
artifact_path = "model/best_model_tfidf.skops"

local_path = mlflow.artifacts.download_artifacts(
    run_id=run_id, artifact_path=artifact_path
)
loaded_model = sio.load(local_path)
loaded_model

## Register the model in MLflow
We re-log the loaded model using MLflow's sklearn flavor so the registry can manage versions.

In [ ]:
registered_model_name = "scam-detector"  # change as needed
with mlflow.start_run(run_name="register_scam_detector"):
    mlflow.sklearn.log_model(
        loaded_model,
        artifact_path="model",
        registered_model_name=registered_model_name,
    )
    latest_uri = mlflow.get_artifact_uri("model")
    print("Logged model at:", latest_uri)
    print("Registered name:", registered_model_name)

## Load the registered model (by version)
Replace `1` with the version you want. Use "latest" to get the newest.

In [ ]:
model_uri = f"models:/{registered_model_name}/1"
registered_model = mlflow.sklearn.load_model(model_uri)
registered_model